In [11]:
import tensorflow as tf
import keras
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, f1_score

import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

In [2]:
tf.__version__

'2.0.0'

In [3]:
from tensorflow.keras import layers
from tensorflow.keras import layers
import datetime
logdir = "logs/scalars/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)


model = tf.keras.Sequential()
model.add(layers.Dense(400,input_shape=(9,), activation='relu'))
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dense(2, activation = 'softmax'))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [5]:
class tune_param_classifier:
    def __init__(self, x, y, epoch,learning_rate, no_of_layers, 
                 batch_size,validation_data):
        
        self.x = x
        self.y = y
        self.epoch = epoch
        self.learning_rate = learning_rate
        self.no_of_layers = no_of_layers
        self.batch_size = batch_size
        self.validation_data = validation_data
        
        self.x_dimension = x.shape
        self.target_dimension = y.shape
        
        #self layers range (start, stop, step)
        self.layers_range = (100,1000,50)
        
    
        #no of possiblities
        self.possibilities = self.no_of_possibilities(len(self.layers_range))
        
        #call evaluate classifier to evaluate the network
                
        return
    
    
    def evaluate_classifier(self):
        # evaluate network
        a = self.x
        b = self.y
        c = self.epoch
        d = self.no_of_layers
        e = self.learning_rate
        f = self.batch_size
        g = self.validation_data
        
        dict_scores = self.evaluate_network(a,b,c,d,e,f,g)
        
        #find param with minimum variance
        result = self.minimum_variance(dict_scores)
        return result
    
    
    def minimum_variance(self, dict_scores):
        variances = [np.var([np.var(m),np.var(n)]) for m,n in zip(dict_scores[i]['training_values'],
                                                                 dict_scores[i]['testing_values']) 
                    for i in dict_scores.keys()]
        result = {}
        for key, value in zip(dict_scores.keys(),variances):  
            if value == min(variances):
                result = dict_scores[key]
                break
        return result
    
    
    def no_of_posibilities(self, no_of_layers):
        count = 1
        for i in range(1, no_of_layers+1):
            count *= i
        return count
    
    
    def create_network(self, neurons):
        model = tf.keras.Sequential()
        
        # iterate through the dict layers
        model.add(layers.Dense(neurons[0],input_shape=(self.x_dimension[1],), activation='relu'))
        
        # add hidden layers
        for i in range(neurons[1:]):
            model.add(layers.Dense(i, activation='relu'))
            
        # add output layers
        if self.target_dimension[1] == None:
            model.add(layers.Dense(1, activation = 'softmax'))
        else:
            model.add(self.target_dimension[1], activation = 'softmax')
            
        return model
    
   
    
    def categorical_compute(self, actual, prediction):
        prediction = tf.argmax(logits, 1)
        actual = tf.argmax(y,1)
        
        TP = tf.math.count_nonzero(prediction * actual)
        TN = tf.math.count_nonzero((prediction - 1) * (actual - 1))
        FP = tf.math.count_nonzero(prediction * (actual - 1))
        FN = tf.math.count_nonzero((prediction - 1) * actual)
        
        accuracy = (TP+TN)/(TP+TN+FP+FN)
        Recall = TP/(TP+FN)
        precision = TP/(TP+FP)
        F1_Score = 2*(Recall * precision) / (Recall + precision)
        return accuracy, f1_score, precision,recall

    
    
    def binary_compute(self, actual, prediction):
        accuracy = accuracy_score(actual, prediction)
        f1_score = f1_score(actual, prediction)
        precision = precision_score(actual, prediction)
        recall = (precision *f1_score)/((2*precision)-f1_score)
        return accuracy, f1_score, precision, recall
    
    
    
    
    def evaluate_network(self,x,y,epoch,no_of_layers,
                         learning_rate,batch_size, 
                         validation_data):
        # ranges of neuron values
        data = [i for i in range(self.layers_range)]
        
        #optimizers
        optimize = {
            'rmsprop':tf.keras.optimizers.RMSprop(learning_rate),
            'gradient descent':tf.keras.optimizers.SGD(learning_rate),
            'adam':tf.keras.optimizers.Adam(learning_rate),
            'adagrad':tf.keras.optimizers.AdaGrad(learning_rate),
            'adadelta':tf.keras.optimizers.AdaDelta(learning_rate),
            'nadem':tf.keras.optimizers.Nadem(learning_rate)
        }
        
        
        # losses
        losses = {
            'binary':tf.keras.losses.binary_crossentropy(from_logits = True),
            'categorical':tf.keras.losses.CategoricalCrossentropy(from_logits = True)
        }
        
        

        dict_neurons = {}
        for pos in range(self.possibilities):
            #randomly initialize values as numbers of neurons
            current_neurons = np.random.choice(data, no_of_layers, replace = False)
            
            
            #create network for the neurons
            model = self.create_network(current_neurons)
            
            #Train the network on available optimizers and losses
            for m in optimize.keys():
                #document training params
                dict_doc = {}
                if (y.shape[1] in [None, 1]):
                    model.compile(optimizer=optimize[m],
                                  loss= losses['binary'],
                                  metrics=['accuracy'])
                    model.fit(x,y, epoch = epoch, batch_size= batch_size,
                             validation_data = validation_data, verbose = 0)
                    pred_train, pred_test = model.predict(x), model.predict(validation_data[0])
                    
                    train_result = self.binary_compute(y, pred_train)
                    test_result = self.binary_compute(validation_data[1],pred_test)
                    
                    # document training params
                    dict_doc['no_of_layers'] = no_of_layers
                    dict_doc['neuron_values'] = current_neurons
                    dict_doc['optimizer'] = m
                    dict_doc['loss'] = 'binary_crossentropy'
                    dict_doc['training_values'] = train_result
                    dict_doc['testing_values'] = test_result
                    
                else:
                    model.compile(optimizer = optimize[m],
                                  loss= losses['categorical'],
                                  metrics=['accuracy'])
                    h = model.fit(x,y, epoch = epoch, batch_size= batch_size,
                             validation_data = validation_data,verbose =0)
                    pred_train, pred_test = model.predict(x), model.predict(validation_data[0])
                    
                    train_result = self.categorical_compute(y, pred_train)
                    test_result = self.categorical_compute(validation_data[1],pred_test)
                    
                    # document training params
                    dict_doc['no_of_layers'] = no_of_layers
                    dict_doc['neuron_values'] = current_neurons
                    dict_doc['optimizer'] = m
                    dict_doc['loss'] = 'binary_crossentropy'
                    dict_doc['training_values'] = train_result
                    dict_doc['testing_values'] = test_result
               
                # take record of all iterations
                dict_neurons[str(pos)+'_'+m] = dict_doc
                    
            
        return dict_neurons

# Regression

In [12]:
class tune_param_classifier:
    def __init__(self, x, y, epoch,learning_rate, no_of_layers, 
                 batch_size,validation_data):
        
        self.x = x
        self.y = y
        self.epoch = epoch
        self.learning_rate = learning_rate
        self.no_of_layers = no_of_layers
        self.batch_size = batch_size
        self.validation_data = validation_data
        
        self.x_dimension = x.shape
        self.target_dimension = y.shape
        
        #self layers range (start, stop, step)
        self.layers_range = (100,1000,50)
        
    
        #no of possiblities
        self.possibilities = self.no_of_possibilities(len(self.layers_range))
        
        #call evaluate classifier to evaluate the network
                
        return
    
    
    def evaluate_classifier(self):
        # evaluate network
        a = self.x
        b = self.y
        c = self.epoch
        d = self.no_of_layers
        e = self.learning_rate
        f = self.batch_size
        g = self.validation_data
        
        dict_scores = self.evaluate_network(a,b,c,d,e,f,g)
        
        #find param with minimum variance
        result = self.minimum_variance(dict_scores)
        return result
    
    
    def minimum_variance(self, dict_scores):
        variances = [np.var([np.var(m),np.var(n)]) for m,n in zip(dict_scores[i]['training_values'],
                                                                 dict_scores[i]['testing_values']) 
                    for i in dict_scores.keys()]
        result = {}
        for key, value in zip(dict_scores.keys(),variances):  
            if value == min(variances):
                result = dict_scores[key]
                break
        return result
    
    
    def no_of_posibilities(self, no_of_layers):
        count = 1
        for i in range(1, no_of_layers+1):
            count *= i
        return count
    
    
    def create_network(self, neurons):
        model = tf.keras.Sequential()
        
        # iterate through the dict layers
        model.add(layers.Dense(neurons[0],input_shape=(self.x_dimension[1],), activation='relu'))
        
        # add hidden layers
        for i in range(neurons[1:]):
            model.add(layers.Dense(i, activation='relu'))
            
        # add output layers
        if self.target_dimension[1] == None:
            model.add(layers.Dense(1, activation = 'softmax'))
        else:
            model.add(self.target_dimension[1], activation = 'softmax')
            
        return model
    
   
    
    def _compute(self, actual, prediction):
        prediction = tf.argmax(logits, 1)
        actual = tf.argmax(y,1)
        
        TP = tf.math.count_nonzero(prediction * actual)
        TN = tf.math.count_nonzero((prediction - 1) * (actual - 1))
        FP = tf.math.count_nonzero(prediction * (actual - 1))
        FN = tf.math.count_nonzero((prediction - 1) * actual)
        
        accuracy = (TP+TN)/(TP+TN+FP+FN)
        Recall = TP/(TP+FN)
        precision = TP/(TP+FP)
        F1_Score = 2*(Recall * precision) / (Recall + precision)
        return accuracy, f1_score, precision,recall
    
    
    
    
    def evaluate_network(self,x,y,epoch,no_of_layers,
                         learning_rate,batch_size, 
                         validation_data):
        # ranges of neuron values
        data = [i for i in range(self.layers_range)]
        
        #optimizers
        optimize = {
            'rmsprop':tf.keras.optimizers.RMSprop(learning_rate),
            'gradient descent':tf.keras.optimizers.SGD(learning_rate),
            'adam':tf.keras.optimizers.Adam(learning_rate),
            'adagrad':tf.keras.optimizers.AdaGrad(learning_rate),
            'adadelta':tf.keras.optimizers.AdaDelta(learning_rate),
            'nadem':tf.keras.optimizers.Nadem(learning_rate)
        }
        
        
        # losses
        losses = {
            'mse':tf.keras.losses.MSE,
            'mae':tf.keras.losses.MAE,
            'mape':tf.keras.losses.MAPE,
        }
        
        

        dict_neurons = {}
        for pos in range(self.possibilities):
            #randomly initialize values as numbers of neurons
            current_neurons = np.random.choice(data, no_of_layers, replace = False)
            
            
            #create network for the neurons
            model = self.create_network(current_neurons)
            
            #Train the network on available optimizers and losses
            for m in optimize.keys():
                #document training params
                dict_doc = {}
                model.compile(optimizer=optimize[m],
                              loss= losses['mae'],
                              metrics=['accuracy'])
                model.fit(x,y, epoch = epoch, batch_size= batch_size,
                         validation_data = validation_data, verbose = 0)
                pred_train, pred_test = model.predict(x), model.predict(validation_data[0])

                train_result = self._compute(y, pred_train)
                test_result = self._compute(validation_data[1],pred_test)

                # document training params
                dict_doc['no_of_layers'] = no_of_layers
                dict_doc['neuron_values'] = current_neurons
                dict_doc['optimizer'] = m
                dict_doc['loss'] = 'mean squared error'
                dict_doc['training_values'] = train_result
                dict_doc['testing_values'] = test_result
                  
                # take record of all iterations
                dict_neurons[str(pos)+'_'+m] = dict_doc
                    
            
        return dict_neurons

Help on function var in module numpy:

var(a, axis=None, dtype=None, out=None, ddof=0, keepdims=<no value>)
    Compute the variance along the specified axis.
    
    Returns the variance of the array elements, a measure of the spread of a
    distribution.  The variance is computed for the flattened array by
    default, otherwise over the specified axis.
    
    Parameters
    ----------
    a : array_like
        Array containing numbers whose variance is desired.  If `a` is not an
        array, a conversion is attempted.
    axis : None or int or tuple of ints, optional
        Axis or axes along which the variance is computed.  The default is to
        compute the variance of the flattened array.
    
        .. versionadded:: 1.7.0
    
        If this is a tuple of ints, a variance is performed over multiple axes,
        instead of a single axis or all the axes as before.
    dtype : data-type, optional
        Type to use in computing the variance.  For arrays of integer t

In [15]:
np.var([np.var([0.55,0.34,0.45,0.90]),np.var([0.55,0.34,0.45,0.90])])

0.0